In [ ]:
import pandas as pd
import numpy as np
import requests

# 해외주식데이터 가져오기

In [221]:
oss=pd.read_csv("oss_stk_orr.csv") #해외주식거래내역
oss.columns=['계좌번호(가명화)',
'주문날짜',
'주문순서',
'주문접수시간대',
'최종체결시간대',
'종목코드',
'매매구분코드',
'체결건수',
'체결외화단가',
'주문매체구분코드',
'거래통화코드',
'거래통화환율'
]

In [317]:
#종목코드(isin) 고유값만 저장
isin=oss['종목코드'].unique()

In [318]:
len(isin)

1277

In [320]:
isin1=isin[:600]
isin2=isin[600:]

# isin ticker mapping 데이터 생성

In [33]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [34]:
import requests

In [ ]:
import datetime
import time

In [225]:
#isin코드를 넣으면 open figi에서 api를 요청해 받아오는 메서드
import requests
import json
def get_ticker(value):
    mykey='07d5a2d7-ffb9-4f03-b518-12564d9e3630' #openFIGI에서 api를 요청할 api key 값
    url = 'https://api.openfigi.com/v3/mapping'
    headers = {'Content-Type':'text/json', 'X-OPENFIGI-APIKEY':mykey }
    payload = [{"idType": "ID_ISIN", "idValue": value}]
    request = requests.post(url, headers=headers, data=json.dumps(payload))
    return request.json()


In [323]:
#isin 리스트를 넣으면 api요청 메서드를 사용해 ticker 매칭 정보를 받아오고 name, ticker, isin을 데이터프레임에 붙여주는 메서드
def isin_data(isin_list):
    isin_ticker_map=pd.DataFrame(columns=['isin','name','ticker'])
    i=0
    start=datetime.datetime.now()
    for code in isin_list:
        isin_df=pd.DataFrame.from_dict(get_ticker(code)[0].get('data'))
        isin_df.insert(0,'isin',code)
        if len(isin_df)==0:
            isin_df.loc[0]=code
            isin_df[['name', 'ticker']]=np.NaN
        isin_df=isin_df[['isin','name','ticker']]
        isin_df=isin_df[['isin','name','ticker']].drop_duplicates(keep='first')
        isin_df=isin_df.reset_index(drop=True)
        isin_ticker_map=pd.concat([isin_ticker_map,isin_df],axis=0)
        i+=1
        print(i)
        if i>=100: #open figi api 요청은 1분에 100개 한정이므로 1분 안에 100개를 불러오면 1분이 끝날때까지 휴식하는 메서드
            end=datetime.datetime.now()
            if (end-start).total_seconds()<=65:
                time.sleep(65-round((end-start).total_seconds()))
            else:
                pass
            start=datetime.datetime.now()
            i=0
        else:
            pass
    return isin_ticker_map

In [321]:
map1=isin_data(isin1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
4

In [322]:
map1

,isin,name,ticker
0,US5494981039,LUCID GROUP INC,LCID
15,US5494981039,LUCID GROUP INC,CCIVGBP
37,US5494981039,LUCID GROUP INC,CCIVUSD
58,US5494981039,LUCID GROUP INC,CH2A
61,US5494981039,LUCID GROUP INC,CCIVEUR
...,...,...,...
29,US62914V1061,NIO INC - ADR,0A1K
30,US62914V1061,NIO INC - ADR,NIOGBX
51,US62914V1061,NIO INC - ADR,NIOMXN
79,US62914V1061,NIO INC - ADR,NIORUB


In [324]:
map2=isin_data(isin2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
4

In [325]:
map2

,isin,name,ticker
0,US3453708600,FORD MOTOR CO,F
1,US3453708600,FORD MOTOR CO,FMC1
2,US3453708600,FORD MOTOR CO,F*
3,US3453708600,FORD MOTOR CO,F1EUR
4,US3453708600,FORD MOTOR CO,F1CHF
...,...,...,...
0,CNE100002TP9,PROYA COSMETICS CO LTD-A,603605
0,US01675A1097,ALLBIRDS INC-CL A,BIRD
1,US01675A1097,ALLBIRDS INC-CL A,BIRDGBP
2,US01675A1097,ALLBIRDS INC-CL A,BIRDUSD


In [326]:
ticker_map=pd.concat([map1,map2],axis=0)

In [327]:
ticker_map

,isin,name,ticker
0,US5494981039,LUCID GROUP INC,LCID
15,US5494981039,LUCID GROUP INC,CCIVGBP
37,US5494981039,LUCID GROUP INC,CCIVUSD
58,US5494981039,LUCID GROUP INC,CH2A
61,US5494981039,LUCID GROUP INC,CCIVEUR
...,...,...,...
0,CNE100002TP9,PROYA COSMETICS CO LTD-A,603605
0,US01675A1097,ALLBIRDS INC-CL A,BIRD
1,US01675A1097,ALLBIRDS INC-CL A,BIRDGBP
2,US01675A1097,ALLBIRDS INC-CL A,BIRDUSD


In [328]:
ticker_map.to_csv("ticker_map.csv", index = False)

# 해외주식데이터 가져오기

In [132]:
!pip install -U finance-datareader
import FinanceDataReader as fdr

Defaulting to user installation because normal site-packages is not writeable


In [137]:
glob_market_list=['S&P500', 'NASDAQ', 'NYSE', 'AMEX','SSE', 'SZSE', 'HKEX', 'TSE', 'HOSE']

In [141]:
stock_oss=pd.DataFrame(columns=['Symbol', 'market','Name', 'Sector', 'Industry'])
for m in glob_market_list:
    df_oss=fdr.StockListing(m)
    df_oss['market']=m
    stock_oss=pd.concat([stock_oss,df_oss],axis=0)

100%|███████████████████████████████████████████████████████████████████████████████| 404/404 [00:00<00:00, 676.53it/s]


In [144]:
stock_oss.to_csv("stock_oss.csv", index = False)

In [157]:
stock_oss[stock_oss['Symbol']=='CCIVGBP']
stock_oss[stock_oss['Name']=='LUCID GROUP INC']

,Symbol,market,Name,Sector,Industry,IndustryCode


In [160]:
stock_oss

,Symbol,market,Name,Sector,Industry,IndustryCode
0,MMM,S&P500,3M,Industrials,Industrial Conglomerates,NaN
1,AOS,S&P500,A. O. Smith,Industrials,Building Products,NaN
2,ABT,S&P500,Abbott,Health Care,Health Care Equipment,NaN
3,ABBV,S&P500,AbbVie,Health Care,Pharmaceuticals,NaN
4,ABMD,S&P500,Abiomed,Health Care,Health Care Equipment,NaN
...,...,...,...,...,...,...
399,PIT,HOSE,Petrolimex International Trading JSC,NaN,다양한 산업재 도매업,522020
400,DXV,HOSE,Danang Building Material Vicem JSC,NaN,건축 자재,512020
401,FUCVREIT,HOSE,Techcom Vietnam REIT,NaN,다양한 금융서비스,555010
402,FUCTVGF4,HOSE,THIEN VIET GROWTH FUND 4,NaN,NaN,NaN


In [329]:
len(stock_oss['Name'].unique())

18220

In [330]:
stock_oss=stock_oss.reset_index(drop=True)

In [331]:
stock_oss

,Symbol,market,Name,Sector,Industry,IndustryCode
0,MMM,S&P500,3M,Industrials,Industrial Conglomerates,NaN
1,AOS,S&P500,A. O. Smith,Industrials,Building Products,NaN
2,ABT,S&P500,Abbott,Health Care,Health Care Equipment,NaN
3,ABBV,S&P500,AbbVie,Health Care,Pharmaceuticals,NaN
4,ABMD,S&P500,Abiomed,Health Care,Health Care Equipment,NaN
...,...,...,...,...,...,...
18317,PIT,HOSE,Petrolimex International Trading JSC,NaN,다양한 산업재 도매업,522020
18318,DXV,HOSE,Danang Building Material Vicem JSC,NaN,건축 자재,512020
18319,FUCVREIT,HOSE,Techcom Vietnam REIT,NaN,다양한 금융서비스,555010
18320,FUCTVGF4,HOSE,THIEN VIET GROWTH FUND 4,NaN,NaN,NaN


# 해외주식 데이터 ticker에 isin 매핑하기

In [335]:
stock_oss=ticker_map.merge(stock_oss,left_on='ticker',right_on='Symbol',how='inner')

In [338]:
#isin 1277개 중 1200개의 코드가 ticker과 매치됨
stock_oss.drop(columns=['name','ticker'], inplace=True)

In [339]:
stock_oss

,isin,Symbol,market,Name,Sector,Industry,IndustryCode
0,US5494981039,LCID,NASDAQ,Lucid Group Inc,NaN,자동차 및 자동차 부품,531010
1,US46222L1089,IONQ,NYSE,IONQ Inc,NaN,소프트웨어 및 IT서비스,572010
2,CA03879J1003,ABUS,NASDAQ,Arbutus Biopharma Corp,NaN,제약,562010
3,US62526P1093,MULN,NASDAQ,Mullen Automotive Inc,NaN,자동차 및 자동차 부품,531010
4,US31773D1019,FNCH,NASDAQ,Finch Therapeutics Group Inc,NaN,생명과학 및 메디컬 리서치,562020
...,...,...,...,...,...,...,...
1195,NL00150001Q9,STLA,NYSE,Stellantis NV,NaN,자동차 및 자동차 부품,531010
1196,US2128731039,CONX,NASDAQ,CONX Corp Class A,NaN,기타 금융업,556010
1197,CNE100002TP9,603605,SSE,Proya Cosmetics Co Ltd Class A,NaN,"개인, 가정 용품 및 서비스",542010
1198,US01675A1097,BIRD,NASDAQ,Allbirds Inc Class A,NaN,섬유 및 의류,532020


In [340]:
stock_oss.isnull().sum()

isin               0
Symbol             0
market             0
Name               0
Sector          1066
Industry           0
IndustryCode     134
dtype: int64

In [341]:
stock_oss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   isin          1200 non-null   object
 1   Symbol        1200 non-null   object
 2   market        1200 non-null   object
 3   Name          1200 non-null   object
 4   Sector        134 non-null    object
 5   Industry      1200 non-null   object
 6   IndustryCode  1066 non-null   object
dtypes: object(7)
memory usage: 75.0+ KB


In [342]:
stock_oss.to_csv("stock_isin_ticker.csv", index = False)